In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 自定义训练和在线预测

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/sdk-custom-image-classification-online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fcustom%2Fsdk-custom-image-classification-online.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/custom/sdk-custom-image-classification-online.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/sdk-custom-image-classification-online.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概述

本教程演示了如何使用Python的Vertex AI SDK来训练和部署用于在线预测的自定义图像分类模型。

了解更多关于[自定义训练](https://cloud.google.com/vertex-ai/docs/training/custom-training)和[Vertex AI预测](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions)的信息。

### 目标

在本教程中，您将学习如何使用`Vertex AI Training`从 Python 脚本中的 Docker 容器创建自定义训练模型，并学习如何使用`Vertex AI Prediction`通过发送数据对部署的模型进行预测。或者，您也可以使用`gcloud`命令行工具或在 Cloud Console 上在线创建自定义训练模型。

本教程使用以下 Google Cloud ML 服务：

- `Vertex AI Training`
- `Vertex AI Prediction`
- `Vertex AI Model` 资源
- `Vertex AI Endpoint` 资源

执行的步骤包括：

- 为训练 TensorFlow 模型创建一个`Vertex AI`自定义作业。
- 将训练的模型工件上传至一个`Model`资源。
- 创建一个服务`Endpoint`资源。
- 将`Model`资源部署到一个服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`资源。

数据集

本教程使用的数据集是[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)中的[cifar10数据集](https://www.tensorflow.org/datasets/catalog/cifar10)。您将使用的数据集版本已经内置在TensorFlow中。训练好的模型可以预测图像属于十个类别中的哪一类：飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船、卡车。

### 成本

本教程使用 Google Cloud（GCP）的可计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储 价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成成本估算。

开始吧

### 为Python安装Vertex AI SDK和其他所需的软件包.

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud \
                                 google-cloud-storage \
                                 pillow \
                                 numpy

### 重新启动运行时（仅限Colab）

要使用新安装的软件包，您必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核将重新启动。请等到重新启动完成后再继续下一步。⚠️</b>
</div>

### 在Colab中验证您的笔记本环境

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息并初始化Python的Vertex AI SDK

要开始使用Vertex AI，您必须拥有一个现有的Google Cloud项目并[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储存储桶

创建一个存储桶来存储诸如数据集之类的中间文物。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

初始化Vertex AI SDK以在您的项目和相应的存储桶中使用Python。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

导入库并定义常量

In [ ]:
import os

import numpy as np
from PIL import Image

### 配置预建容器

Vertex AI 提供预建容器用于运行训练和预测。

有关最新列表，请参阅[用于训练的预建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)和[用于预测的预建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)。

In [ ]:
TRAIN_VERSION = "tf-cpu.2-9"
DEPLOY_VERSION = "tf2-cpu.2-9"

TRAIN_IMAGE = "us-docker.pkg.dev/vertex-ai/training/{}:latest".format(TRAIN_VERSION)
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(DEPLOY_VERSION)

# 教程

现在您已经准备好使用CIFAR10开始创建自己的定制训练模型了。
## 训练模型

有两种方法可以使用容器映像来训练自定义模型：

- **使用谷歌云预构建的容器**。如果您使用预构建的容器，则还需要指定要安装到容器映像中的Python软件包。这个Python软件包包含了用于训练自定义模型的代码。

- **使用您自己的定制容器映像**。如果您使用自己的容器，则容器需要包含用于训练自定义模型的代码。

### 定义训练脚本的命令参数

准备要传递给训练脚本的命令行参数。
- `args`：要传递给相应Python模块的命令行参数。在这个例子中，它们将是：
  - `"--epochs=" + EPOCHS`：训练的纪元数。
  - `"--steps=" + STEPS`：每一纪元的步数（批次）。
  - `"--distribute=" + TRAIN_STRATEGY"`：用于单设备或分布式训练的训练分发策略。
     - `"single"`：单设备。
     - `"mirror"`：单个计算实例上的所有GPU设备。
     - `"multi"`：所有计算实例上的所有GPU设备。

In [ ]:
JOB_NAME = "custom_job_unique"
MODEL_DIR = "{}/{}".format(BUCKET_URI, JOB_NAME)

TRAIN_STRATEGY = "single"

EPOCHS = 20
STEPS = 100

CMDARGS = [
    "--epochs=" + str(EPOCHS),
    "--steps=" + str(STEPS),
    "--distribute=" + TRAIN_STRATEGY,
]

#### 训练脚本

在下一个单元格中，您将编写训练脚本 `task.py` 的内容。总结如下：

- 从环境变量 `AIP_MODEL_DIR` 获取保存模型文件的目录。该变量由训练服务设置。
- 从 TF Datasets (tfds) 加载 CIFAR10 数据集。
- 使用 TF.Keras 模型 API 构建模型。
- 编译模型 (`compile()`)。
- 根据参数 `args.distribute` 设置训练分布策略。
- 根据参数 `args.epochs` 和 `args.steps` 训练模型 (`fit()`)。
- 将已训练模型保存到指定模型目录 (`save(MODEL_DIR)`)。

In [ ]:
%%writefile task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64

def make_datasets_unbatched():
  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label

  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model

# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
MODEL_DIR = os.getenv("AIP_MODEL_DIR")

train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(MODEL_DIR)

### 训练模型

在Vertex AI上定义您的自定义训练作业。

使用`CustomTrainingJob`类来定义作业，它接受以下参数：

- `display_name`：此训练流程的用户定义名称。
- `script_path`：训练脚本的本地路径。
- `container_uri`：训练容器映像的URI。
- `requirements`：脚本的Python包依赖项列表。
- `model_serving_container_image_uri`：可为您的模型提供预测的容器的URI - 可以是预构建容器或自定义容器。

使用`run`函数启动训练，接受以下参数：

- `args`：要传递给Python脚本的命令行参数。
- `replica_count`：工作节点副本的数量。
- `model_display_name`：如果脚本生成托管`Model`，则为`Model`的显示名称。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作节点副本的加速器数量。

`run`函数创建一个训练流程，训练并创建一个`Model`对象。训练流程完成后，`run`函数将返回`Model`对象。

In [ ]:
job = aiplatform.CustomTrainingJob(
    display_name=JOB_NAME,
    script_path="task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["tensorflow_datasets"],
    model_serving_container_image_uri=DEPLOY_IMAGE,
)

MODEL_DISPLAY_NAME = "cifar10_unique"

# Start the training
model = job.run(
    model_display_name=MODEL_DISPLAY_NAME,
    args=CMDARGS,
    replica_count=1,
)

### 部署模型

在使用模型进行预测之前，您需要将其部署到一个`Endpoint`上。您可以通过在`Model`资源上调用`deploy`函数来实现这一点。这将执行两个操作：

1. 为部署`Model`资源创建一个`Endpoint`资源。
2. 将`Model`资源部署到`Endpoint`资源上。

该函数接受以下参数：

- `deployed_model_display_name`：部署模型的人类可读名称。
- `traffic_split`：在端点上进入此模型的流量百分比，指定为一个或多个键/值对的字典。
   - 如果只有一个模型，则指定为**{"0": 100}**，其中"0"表示上传的这个模型，100表示100%的流量。
   - 如果端点上有现有模型，需要对流量进行分配，则使用`model_id`指定为**{"0": 百分比, model_id: 百分比, ...}**，其中`model_id`是要部署到端点的现有模型的模型ID。百分比必须加起来等于100。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作程序副本的加速器数。
- `starting_replica_count`：最初提供的计算实例数量。
- `max_replica_count`：要扩展到的计算实例的最大数量。在本教程中，仅会配置一个实例。

### 流量分配

`traffic_split`参数被指定为Python字典。您可以将多个模型实例部署到一个端点，并设置流量分配到每个实例的百分比。

您可以使用流量分配逐渐将新模型引入生产环境。例如，如果您在生产中有一个现有模型占100%的流量，您可以将新模型部署到同一个端点，将10%的流量导向它，将原始模型的流量减少为90%。这样可以在最小程度影响大多数用户的情况下监测新模型的性能。

### 计算实例扩展

您可以指定一个单独的实例（或节点）来提供您的在线预测请求。本教程使用单个节点，因此变量`MIN_NODES`和`MAX_NODES`都设置为`1`。

如果要使用多个节点来提供您的在线预测请求，请将`MAX_NODES`设置为您想要使用的最大节点数。Vertex AI会自动调整用于提供预测的节点数，直到达到您设置的最大数量。请参考[定价页面](https://cloud.google.com/vertex-ai/pricing#prediction-prices)了解使用多个节点进行自动扩展的成本。

### Endpoint

该方法将阻止，直到模型被部署，最终返回一个`Endpoint`对象。如果这是首次将模型部署到端点，则可能需要额外几分钟来完成资源的提供。

In [ ]:
DEPLOYED_NAME = "cifar10_deployed_unique"

TRAFFIC_SPLIT = {"0": 100}

MIN_NODES = 1
MAX_NODES = 1


endpoint = model.deploy(
    deployed_model_display_name=DEPLOYED_NAME,
    traffic_split=TRAFFIC_SPLIT,
    min_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
)

## 发送在线预测请求

向部署的模型发送在线预测请求。
### 获取测试数据

从CIFAR数据集中下载图像并进行预处理。

#### 下载测试图像

从CIFAR数据集中下载提供的图像集。

In [ ]:
# Download the images
! gsutil -m cp -r gs://cloud-samples-data/ai-platform-unified/cifar_test_images .

#### 预处理图片
在将数据传递至端点之前，您需要对其进行预处理，以匹配您在`task.py`中定义的自定义模型所期望的格式。

`x_test`：
通过将每个像素除以255来对像素数据进行归一化（重新缩放）。这将用0到1之间的32位浮点数替换每个单字节整数像素。

`y_test`：
您可以从图像文件名中提取标签。每个图像的文件名格式为“image_{LABEL}_{IMAGE_NUMBER}.jpg”。

In [ ]:
# Load image data
IMAGE_DIRECTORY = "cifar_test_images"

image_files = [file for file in os.listdir(IMAGE_DIRECTORY) if file.endswith(".jpg")]

# Decode JPEG images into numpy arrays
image_data = [
    np.asarray(Image.open(os.path.join(IMAGE_DIRECTORY, file))) for file in image_files
]

# Scale and convert to expected format
x_test = [(image / 255.0).astype(np.float32).tolist() for image in image_data]

# Extract labels from image name
y_test = [int(file.split("_")[1]) for file in image_files]

### 发送预测请求

现在您已经有了测试图片，可以使用它们来发送预测请求。使用`Endpoint`对象的`predict`函数，该函数接受以下参数：

- `instances`：一个图像实例的列表。根据您的自定义模型，每个图像实例应该是一个包含浮点数的3维矩阵。这是在前一步中准备的。

`predict`函数返回一个列表，列表中的每个元素对应请求中的对应图像。您将在每个预测的输出中看到：
- 针对预测的置信度水平（`predictions`），在每个十个类别中的每一个介于0到1之间。

然后，您可以对预测结果进行快速评估：
1. `np.argmax`：将每个置信度列表转换为标签
2. 比较预测的标签与实际标签
3. 计算`准确率`为`正确/总数`。

In [ ]:
predictions = endpoint.predict(instances=x_test)
y_predicted = np.argmax(predictions.predictions, axis=1)

correct = sum(y_predicted == np.array(y_test))
accuracy = len(y_predicted)
print(
    f"Correct predictions = {correct}, Total predictions = {accuracy}, Accuracy = {correct/accuracy}"
)

取消部署模型

要从正在提供服务的`Endpoint`资源中取消部署您的`Model`资源，请使用端点的 `undeploy` 方法，并使用以下参数：

- `deployed_model_id`：在部署`Model`资源时由端点服务返回的模型部署标识符。您可以使用端点的 `deployed_models`属性检索已部署的模型。

由于这是`Endpoint`资源上唯一部署的模型，您可以省略`traffic_split`。

In [ ]:
deployed_model_id = endpoint.list_models()[0].id
endpoint.undeploy(deployed_model_id=deployed_model_id)

清理工作

要清理此项目中使用的所有Google Cloud资源，可以[删除用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 训练作业
- 模型
- 终端
- 云存储桶

In [ ]:
# Warning: Setting this to true will delete everything in your bucket
delete_bucket = False

# Delete the training job
job.delete()

# Delete the model
model.delete()

# Delete the endpoint
endpoint.delete()

if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI